In [ ]:
!pip install gymnasium[atari] stable-baselines3[extra] torch tensorboard ale-py
!pip install jupyter matplotlib


In [ ]:
import gymnasium as gym
import ale_py
import numpy
# Create Breakout environment with frame skip & preprocessing
from gymnasium.wrappers import FrameStackObservation,AtariPreprocessing
from stable_baselines3.common.callbacks import BaseCallback

class TrackProgressCallback(BaseCallback):
    def __init__(self, save_freq=100_000, verbose=1):
        super().__init__(verbose)
        self.episode_rewards = []
        self.avg_rewards = []
        self.save_freq = save_freq

    def _on_step(self):
        # Track episode reward
        if len(self.locals["infos"]) > 0 and "episode" in self.locals["infos"][0]:
            r = self.locals["infos"][0]["episode"]["r"]
            self.episode_rewards.append(r)
            # Running mean for last 100 episodes
            avg = np.mean(self.episode_rewards[-100:]) if len(self.episode_rewards) > 0 else r
            self.avg_rewards.append(avg)
        # Save weights at intervals
        if self.num_timesteps % self.save_freq == 0:
            self.model.save(f"dqn_breakout_step_{self.num_timesteps}")
        return True

    def _on_training_end(self):
        np.save("episode_rewards.npy", np.array(self.episode_rewards))
        np.save("average_rewards.npy", np.array(self.avg_rewards))

env = gym.make("ALE/Breakout-v5", render_mode="rgb_array",frameskip=1)
env = AtariPreprocessing(env, terminal_on_life_loss=True)
env = FrameStackObservation(env, stack_size=4)


In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

model = A2C(
    "CnnPolicy",
    env,
    learning_rate=7e-4,                # Typical for Atari, can tune between 2.5e-4 and 7e-4[web:30][web:31]
    n_steps=5,                         # Default for A2C, but 128 is also used for PPO; for A2C, keep it low[web:31]
    gamma=0.99,                        # Standard for Atari[web:30][web:31]
    ent_coef=0.01,                     # Entropy bonus for exploration[web:31]
    vf_coef=0.5,                       # Value function coefficient[web:31]
    max_grad_norm=0.5,                 # Gradient clipping[web:31]
    rms_prop_eps=1e-5,                 # RMSprop epsilon[web:31]
    use_rms_prop=True,                 # Use RMSprop optimizer[web:31]
    policy_kwargs=dict(
        optimizer_class=RMSpropTFLike,
        optimizer_kwargs=dict(eps=1e-5)
    ),
    tensorboard_log="./tensorboard_log_a2c/",
    verbose=1,
    device="cuda"                      # Use GPU if available[web:31]
)

progress = TrackProgressCallback(save_freq=100_000)
# Train for 1 million steps (adjust if needed)
model.learn(total_timesteps=3_000_000)


model.save("a2c_breakout")
print("Finished training and tracking progress.")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard_log_a2c/A2C_1


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Streaming output truncated to the last 5000 lines.
|    value_loss         | 1.8e-05  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 44.4     |
|    ep_rew_mean        | 0.49     |
| time/                 |          |
|    fps                | 86       |
|    iterations         | 30600    |
|    time_elapsed       | 1773     |
|    total_timesteps    | 153000   |
| train/                |          |
|    entropy_loss       | -1.14    |
|    explained_variance | -8.15    |
|    learning_rate      | 0.0007   |
|    n_updates          | 30599    |
|    policy_loss        | -0.114   |
|    value_loss         | 0.0084   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 46       |
|    ep_rew_mean        | 0.53     |
| time/                 |          |
|    fps                | 86       |
|    iterations         

In [ ]:
import matplotlib.pyplot as plt
ep_rewards = np.load("episode_rewards.npy")
avg_rewards = np.load("average_rewards.npy")

plt.plot(avg_rewards)
plt.xlabel("Episode")
plt.ylabel("100-episode Average Reward")
plt.title("Breakout DQN Learning Curve")
plt.show()

# Evaluation (can also record video as in your code block)
env = gym.make("ALE/Breakout-v5", render_mode="rgb_array",frameskip=1)
env = AtariPreprocessing(env, terminal_on_life_loss=True)
env = FrameStackObservation(env, stack_size=4)

obs, _ = env.reset()
done, truncated = False, False
total_reward = 0
while not (done or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    total_reward += reward
print("Eval total reward:", total_reward)

In [ ]:
obs, _ = env.reset()
done, truncated = False, False
total_reward = 0

while not (done or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    total_reward += reward

print("Total reward from PPO agent:", total_reward)


In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

eval_env = DummyVecEnv([lambda: gym.make("ALE/Breakout-v5", render_mode="rgb_array")])
eval_env = VecVideoRecorder(
    eval_env,
    "./videos_ppo/",
    record_video_trigger=lambda step: True,
    video_length=1000,
    name_prefix="ppo-breakout"
)

obs = eval_env.reset()
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, rewards, dones, infos = eval_env.step(action)
eval_env.close()
